In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('./data.txt',delimiter = '\t')

In [2]:
class multipul_leaner_regression():
    def __init__(self, w = 0, intercept = 1):
        self.w = w
        self.intercept = intercept
    
    def preprocessing(self, data):
        if self.intercept == 1:
            data['y절편'] = 1
        data.drop(['Id'], axis = 1, inplace = True)
        return data

    def slicing(self, x_array, p):

        tr_data = x_array.sample(frac = p, random_state = 100)
        
        ts_data = x_array.drop(tr_data.index)
        
        return tr_data, ts_data
    
    def learn(self, data):
        x = []
        y = []
        array = data.values
        for i in range(len(array)):
            y.append(array[i][0])
            x.append(array[i][1:])
        x = np.array(x)
        y = np.array(y)
        self.w = np.linalg.inv(x.T @ x) @ x.T @ y
        return self.w
        
    
    def test(self, data):
        array = data.values
        x = []
        y = []
        pred_y = []
        pred_y = np.array(pred_y)
        
        for i in range(len(array)):
            x.append(array[i][1:])
            y.append(array[i][0])
        
        x = np.array(x)
        y = np.array(y)

        pred_y = np.dot(x, self.w)
        pred_y = pred_y.astype(np.int64)
        y_sub = y - pred_y
        y_sub = pd.DataFrame(y_sub)
        pred_y = pd.DataFrame(pred_y)
        y = pd.DataFrame(y)
        
        test_data = pd.concat([pred_y, y,y_sub], axis = 1)
        test_data.columns = ['예측값','실제값','차이값']
        return test_data
            
class Metric:
    def MSE(self, data):
        array = data.values
        pred_y = []
        y = []
        y_sub = []
        for i in range(len(array)):
            pred_y.append(array[i][0])
            y.append(array[i][1])
            y_sub.append(array[i][2])
        y_sub = np.array(y_sub)    
        pred_y = np.array(pred_y)
        y = np.array(y)
        mse_e = sum((y_sub)**2)/len(y)
        return mse_e
    
    def MAPE(self, data):
        array = data.values
        pred_y = []
        y = []
        y_sub = []    
        
        for i in range(len(array)):
            pred_y.append(array[i][0])
            y.append(array[i][1])
            y_sub.append(array[i][2])
        
        pred_y = np.array(pred_y)
        y = np.array(y)
        y_sub = np.array(y_sub)   
        
        mape_e = sum(abs(y_sub)/abs(y))/len(y) * 100
        return mape_e

In [96]:
l = multipul_leaner_regression()
m = Metric()

p = 0.75 #(75 : 25)

data_pre = l.preprocessing(data) #전처리
slicing = l.slicing(data_pre, p) #training test파일 분할

In [97]:
w = l.learn(slicing[0]) # w 값 학습
t = l.test(slicing[1]) # 실제값과 예측값 비교
t #예측값과 실제값 비교 dataframe

,예측값,실제값,차이값
0,16734,13500,-3234
1,16732,13950,-2782
2,17533,19950,2417
3,16901,21500,4599
4,16340,17950,1610
...,...,...,...
354,8387,8750,363
355,8399,7500,-899
356,8210,7750,-460
357,9039,8950,-89


In [98]:
mse = m.MSE(t) #MSE
mape = m.MAPE(t) #MAPE
print('MSE : {:.1f}\nMAPE : {:.1f}%'.format(mse, mape))

MSE : 2464375.3
MAPE : 10.2%
